In [ ]:
import h5py
import numpy as np

# Load RadCal MATLAB v7.3 file containing FM CAL EXT data:
mat_file_path = './calibration-data/RadCal.mat'
radcal_dict = {}

# First, determine the structure of "RadCal.mat":
with h5py.File(mat_file_path, 'r') as f:
    radcal_data = f['RadCal'][:]         # Load all cell references
    ref = radcal_data[0, 0]              # Get reference to the first calibration struct

    if isinstance(ref, h5py.Reference):  # Check if it's a valid struct reference
        struct_data = f[ref]             # Dereference struct

        print(f"\nCell {1} contains struct with fields:", list(struct_data.keys()))

        # Extract datasets from each subfield
        for field in ['Cal', 'Cold', 'Hot', 'Int']:
            if field in struct_data:
                print(f"  {field} contains:", list(struct_data[field].keys()))  # Show datasets inside


# NOW, knowing RadCal strucutre, extract the data into an HDF5 file & Python dictionary to confirm data:
with h5py.File(mat_file_path, 'r') as f:
    radcal_data = f['RadCal'][:]    

    # Loop over 31 cells:
    for i in range(radcal_data.shape[0]):  
        ref = radcal_data[i, 0] 

        if isinstance(ref, h5py.Reference):  
            struct_data = f[ref]            

            # Initialize storage for this cell
            radcal_dict[f"Cell_{i+1}"] = {}

            # Create a new HDF5 file for this cell
            hdf5_filename = f"./calibration-data/ltm-cal-ext-{i+1}.h5"
            with h5py.File(hdf5_filename, 'w') as hdf5_file:
                print(f"Creating file: {hdf5_filename}")
            
                # Loop through the 4 main fields: Cal, Cold, Hot, Int
                for category in ['Cal', 'Cold', 'Hot', 'Int']:
                    if category in struct_data:
                        radcal_dict[f"Cell_{i+1}"][category] = {}
                        cat_group = hdf5_file.create_group(category)

                        # Loop through datasets inside each category
                        for dataset_name in struct_data[category].keys():
                            item = struct_data[category][dataset_name]
                            
                            # Check if the item is a dataset (array)
                            if isinstance(item, h5py.Dataset):
                                dataset = item[:]  
                                cat_group.create_dataset(dataset_name, data=dataset)
                                radcal_dict[f"Cell_{i+1}"][category][dataset_name] = dataset

                            # If it's another reference, print a warning (for debugging)
                            elif isinstance(item, h5py.Group):
                                print(f"Skipping subgroup '{dataset_name}' in {category} (it's a nested group)")
                
            print(f"Extracted Cell {i+1}")
            print(f"Saved Cell {i+1} to {hdf5_filename}")
    
# To inspect groups of HDF5 files: 
print("\n\nPrinting Groups of ltm-cal-ext-15.h5 file to verify: \n")
with h5py.File("./calibration-data/ltm-cal-ext-1.h5", "r") as f:
    print("Groups:", list(f.keys()))  
    print("Cold datasets:", list(f["Cold"].keys()))  


In [ ]:
# Access the python dict with data: 
radcal_dict["Cell_10"]["Cold"]["frames"]


In [ ]:
Cold contains: ['BBtemp', 'BBtempNoise', 'ChannelRadiance', 'ColdBBTemp', 'DeltaFrame', 'GAIN', 'HKTemp', 'HKTempChange', 'HKTempNoise', 'HKdataEnd', 'HKdataStart', 'HotBBTemp', 'InstrTemp', 'LTMprops', 'OGSETempMean', 'OGSETempNoise', 'OGSETemps', 'RadianceFrame', 'SceneTemp', 'VCSL', 'VID_PW', 'average', 'corAverage', 'corFrames', 'corNoise', 'dacTable', 'frames', 'hasDAC', 'noise', 'numFrames', 'offsetAverage', 'stableCorAverage', 'stableCorFrames', 'stableCorNoise', 'startTime', 'target']